El objetivo de esta notebook es el enriquecimiento del data de propiedades con la fusión con Argentina Points of Interest 

## Cargar el archivo KML

In [1]:
import xml.etree.ElementTree as ET

# Cargar el archivo KML
tree = ET.parse("hotosm_arg_points_of_interest_points_kml.kml")
root = tree.getroot() 
#getroot()obtiene el nodo raíz de KML para comenzar a trabajar con los datos

In [2]:
#indicación para decirle a Python: busca dentro del grupo de etiquetas KML
namespace = {"kml": "http://www.opengis.net/kml/2.2"}

In [3]:
# inicializo para ir guardando info
name = []
lat = []
lon = []
# Buscar todos los puntos <Placemark>
for placemark in root.findall(".//kml:Placemark", namespace):
    latlon = placemark.find(".//kml:coordinates", namespace)
    try:
        lat_, lon_ = latlon.text.strip().split(',')
        name_ = placemark.find("kml:name", namespace).text

        lat.append(lat_)
        lon.append(lon_)
        name.append(name_)
    except AttributeError:
        pass


### Creación dataframe con los POIS: df_pois

In [4]:
import pandas as pd
#crear un dataframe con los POIS
df_pois= pd.DataFrame({'name':name ,'lat':lat, 'lon':lon})

print(df_pois.head())

                            name          lat          lon
0                          bench  -60.5445411  -33.8918493
1           Estancia San Lorenzo  -63.8917999  -42.1324351
2                          bench  -60.5445154  -33.8918687
3  Nuevo Banco Del Chaco Makalle  -59.2864208  -27.2083748
4                       Estancia  -63.3819324  -40.6975211


In [5]:
#chequear si los POIS estan en Argentina
df_pois[['lat','lon']].describe()

,lat,lon
count,246528,246528
unique,243924,242931
top,-69.129,-45.827
freq,7,15


Insight:
Con los valores estadisticos de lat y lon, podemos verificar que si los puntos están dentro de Argentina.

 Coordenadas de Argentina (aproximadas):
 
Longitud (lon): de -73.5 (oeste, límite con Chile) hasta -53.6 (este, Misiones)

Latitud (lat): de -55.1 (sur, Tierra del Fuego) hasta -21.8 (norte, Jujuy)

In [6]:
df_pois.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246528 entries, 0 to 246527
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   name    246528 non-null  object
 1   lat     246528 non-null  object
 2   lon     246528 non-null  object
dtypes: object(3)
memory usage: 5.6+ MB


In [7]:
#conversión  lat y lon a float
df_pois['lat']= pd.to_numeric(df_pois['lat'], errors='coerce')
df_pois['lon']= pd.to_numeric(df_pois['lon'], errors='coerce')

print(df_pois.dtypes)

name     object
lat     float64
lon     float64
dtype: object


In [8]:
df_pois.name.value_counts().head(10)

name
bench             10560
waste_basket       2452
parking            1865
fuel               1754
toilets            1701
atm                1392
shelter            1188
waste_disposal     1097
pharmacy            948
YPF                 933
Name: count, dtype: int64

In [9]:
df_pois.nunique()

name    163691
lat     243924
lon     242931
dtype: int64

In [10]:
df_pois.isnull().sum()

name    0
lat     0
lon     0
dtype: int64

In [11]:
#exploración colegios
df_pois[df_pois.name.str.lower().str.contains('escuela|colegio')]

,name,lat,lon
11,Colegio General Martín Miguel de Güemes,-63.514570,-36.155606
15,Agrupamiento 86135 Escuela 811,-64.054212,-28.215283
20,Escuela Mariano Moreno,-64.043461,-31.046330
27,J.I.N. 13 en Escuela 133,-63.586801,-37.458493
33,Escuela Publica Leonila de las Meredes Lemos,-63.978015,-33.280378
...,...,...,...
246289,Escuela,-64.572734,-32.175220
246322,Comedor escolar de la Escuela 504,-61.334240,-33.800066
246328,"Escuela N° 1250 ""Almirante Guillermo Brown""",-60.701275,-31.632668
246503,Escuela de Educación Primaria N° 17 Juan Bauti...,-60.186197,-33.350238


In [12]:
#exploración universidades
df_pois[df_pois.name.str.lower().str.contains('universidad|facultad')]

,name,lat,lon
1023,Universidad Popular UP Porteña,-62.070498,-31.012439
2907,Universidad Nacional De La Rioja - Sede Chamical,-66.312664,-30.368901
3730,Museo y Archivo Histórico de la Facultad de De...,-58.391535,-34.582599
4118,Universidad Nacional del noreste,-60.613359,-25.944989
5023,Biblioteca Central de la Universidad Nacional ...,-62.270050,-38.701749
...,...,...,...
241234,Buffet de la Facultad de Informática,-57.938256,-34.903678
241257,Facultad Relaciones Internacionales - UM,-68.860824,-32.891743
244085,Universidad CAECE,-58.378736,-34.609007
245222,Universidad Nacional del Comahue,-71.359182,-40.157148


In [13]:
#exploración tren, subte, y colectivos
df_pois[df_pois.name.str.lower().str.contains('subte|tren|colectivo')]

,name,lat,lon
1458,After Office TRENDY Miercoles Palermo,-58.435788,-34.585834
2610,El Tren del Fin del Mundo,-68.423796,-54.833191
3678,Parroquia Castrense Inmaculada Concepción,-55.917151,-27.361687
3717,Tren Museo,-68.848527,-32.883689
7160,Boletería Tren internacional,-55.883905,-27.380325
...,...,...,...
238753,El Tren Mágico Boletería,-56.689690,-36.540511
238966,Parapente la Cuesta del Tren,-64.938582,-32.370769
239734,"Jardín de infantes ""El Trencito Musical""",-59.628438,-35.636357
241694,Playa de estacionamiento subterránea,-64.191713,-31.416834


In [14]:
#exploración de hospitales, clinicas, sanatorios
df_pois[df_pois.name.str.lower().str.contains('hospital|clinica|sanatorio')]

,name,lat,lon
111,hospital,-58.397022,-34.607249
255,Clinica Privada Etruria,-63.245006,-32.941266
303,Hospital Vespucio,-63.860679,-22.585783
305,Hospital General Mosconi,-63.811483,-22.594898
573,Hospital San Martín,-60.506294,-31.731757
...,...,...,...
245626,Nueva Clinica Privada Beccar,-58.529511,-34.460024
245855,Hospital,-65.624475,-33.053862
246121,Clinica San Bernardo,-56.680347,-36.687636
246318,Hospital de Rehabilitación Doctor Vera Candioti,-60.724931,-31.653125


In [15]:
#exploración de parques
df_pois[df_pois.name.str.lower().str.contains('parque|plaza')]


,name,lat,lon
19,Plaza,-63.894945,-27.924300
330,Café de la Plaza,-71.352051,-40.157400
419,Guardaparque,-54.014529,-25.809204
1199,Residencial Plaza,-62.007134,-30.709975
1263,El Parque,-58.536577,-34.455258
...,...,...,...
245315,Plaza 3 Apart,-64.250976,-27.792992
245918,Panadería La Plaza,-71.463880,-43.079875
246059,Kiosco Plaza,-65.104485,-43.299756
246117,Confitería La Plaza,-65.105874,-43.300223


In [16]:
df_pois.to_csv('../data/---external u otras fuentes/df_pois.csv', index=False)

In [17]:
import os

# Verificar si el archivo ya existe
file_path = '../data/---external u otras fuentes/df_pois.csv'

#guardar el csv
if not os.path.exists(file_path):
    df_pois.to_csv(file_path, index= False)
    print('archivo guardado')
else:
    print('el archivo ya exisiste')

el archivo ya exisiste


## Cargar dataset: df_propiedades

In [18]:
import pandas as pd
df_propiedades=pd.read_csv('../data/---datos limpios/df_propiedades.csv')

In [19]:
df_propiedades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 609185 entries, 0 to 609184
Data columns (total 23 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   localidad                609185 non-null  object 
 1   id                       609185 non-null  object 
 2   ad_type                  609185 non-null  object 
 3   start_date               609185 non-null  object 
 4   end_date                 609185 non-null  object 
 5   created_on               609185 non-null  object 
 6   lat                      609185 non-null  float64
 7   lon                      609185 non-null  float64
 8   país                     609185 non-null  object 
 9   provincia                609185 non-null  object 
 10  rooms                    609185 non-null  float64
 11  bedrooms                 609185 non-null  float64
 12  bathrooms                609185 non-null  float64
 13  surface_total            609185 non-null  float64
 14  surf

## Opción 1

Se intentara cruzar df_pois con df_propiedades para encontrar distancias cercanas de hospitales, escuelas, universidades, parques.

In [20]:
import geopandas as gpd
from shapely.geometry import Point

# Paso 1: Crear GeoDataFrames
gdf_propiedades = gpd.GeoDataFrame(
    df_propiedades,
    geometry=gpd.points_from_xy(df_propiedades['lon'], df_propiedades['lat']),
    crs="EPSG:4326"
)

gdf_pois = gpd.GeoDataFrame(
    df_pois,
    geometry=gpd.points_from_xy(df_pois['lon'], df_pois['lat']),
    crs="EPSG:4326"
)

In [21]:
# Paso 2: Convertir a coordenadas métricas para calcular distancia en metros
gdf_propiedades = gdf_propiedades.to_crs("EPSG:3857")
gdf_pois = gdf_pois.to_crs("EPSG:3857")

In [22]:
categorias = {
    'hospital': 'hospital|clínica|sanatorio|salud|centro médico|unidad sanitaria',
    'universidad': 'universidad|facultad|tecnológica|universitario|univ\.|u\.n\.|uca|uba',
    'escuela': 'escuela|colegio|primaria|secundaria|educación|jardín|e\.e\.p\.|e\.e\.s\.|e\.e\.m\.|e\.e\.t\.|e\.n\.s\.|educativo|nivel',
    'parque': 'plaza|parque|reserva|espacio verde',
    # Si luego querés agregar 'transporte', lo agregás acá también
}

<>:3: SyntaxWarning: invalid escape sequence '\.'
<>:4: SyntaxWarning: invalid escape sequence '\.'
<>:3: SyntaxWarning: invalid escape sequence '\.'
<>:4: SyntaxWarning: invalid escape sequence '\.'
C:\Users\NB USER\AppData\Local\Temp\ipykernel_7744\47204136.py:3: SyntaxWarning: invalid escape sequence '\.'
  'universidad': 'universidad|facultad|tecnológica|universitario|univ\.|u\.n\.|uca|uba',
C:\Users\NB USER\AppData\Local\Temp\ipykernel_7744\47204136.py:4: SyntaxWarning: invalid escape sequence '\.'
  'escuela': 'escuela|colegio|primaria|secundaria|educación|jardín|e\.e\.p\.|e\.e\.s\.|e\.e\.m\.|e\.e\.t\.|e\.n\.s\.|educativo|nivel',


Visualización rapida de las coinicidencias

In [23]:
for categoria, pattern in categorias.items():
    coincidencias = gdf_pois[gdf_pois['name'].str.lower().str.contains(pattern, na=False, regex=True)]
    print(f"{categoria}: {len(coincidencias)} coincidencias")


hospital: 3183 coincidencias
universidad: 5135 coincidencias
escuela: 18611 coincidencias
parque: 885 coincidencias


In [24]:
from shapely.strtree import STRtree

for categoria, pattern in categorias.items():
    print(f"Procesando categoría: {categoria}...")

    gdf_categoria = gdf_pois[gdf_pois['name'].str.lower().str.contains(pattern, na=False, regex=True)]

    if gdf_categoria.empty:
        gdf_propiedades[f'distancia_{categoria}_m'] = None
        continue

    geometries = gdf_categoria.geometry.values
    tree = STRtree(geometries)

    # Calcular la distancia mínima, con verificación de lista vacía
    def calcular_distancia_mas_cercana(geom):
        candidatos = tree.query(geom)
        if len(candidatos) == 0:
            return None
        return min(geom.distance(poi) for poi in candidatos)

    distancias = gdf_propiedades.geometry.apply(calcular_distancia_mas_cercana)

    gdf_propiedades[f'distancia_{categoria}_m'] = distancias

    print(f"✓ Completado: distancia_{categoria}_m")

Procesando categoría: hospital...
✓ Completado: distancia_hospital_m
Procesando categoría: universidad...
✓ Completado: distancia_universidad_m
Procesando categoría: escuela...
✓ Completado: distancia_escuela_m
Procesando categoría: parque...
✓ Completado: distancia_parque_m


In [25]:
print(gdf_categoria.geometry.head())
print(gdf_categoria.crs)

19       POINT (-3108518.868 -9323136.753)
330     POINT (-4470301.308 -11523734.588)
419       POINT (-2873067.447 -7172908.45)
1199     POINT (-3418618.824 -8860834.517)
1263      POINT (-3835541.753 -8080890.55)
Name: geometry, dtype: geometry
EPSG:3857


In [26]:
print(gdf_propiedades.geometry.head())
print(gdf_propiedades.crs)

0    POINT (-6531580.508 -4080756.036)
1    POINT (-6526716.737 -4116205.825)
2    POINT (-6522302.841 -4116407.125)
3    POINT (-6531007.269 -4115179.776)
4     POINT (-6528040.876 -4111798.66)
Name: geometry, dtype: geometry
EPSG:3857


In [37]:
gdf_pois['name'].str.lower().sample(20).tolist()


['pizza gigante',
 'centro provincial de salud infantil',
 'grido',
 'ypf.sc.cs-2162',
 'auto parabrisas y cristales',
 'bomberos voluntarios de tartagal',
 'ypf.ch.e-1096(d)',
 'aoc.ch.z-179',
 'parking_entrance',
 'la casa del cd',
 'ypf.sc.cl-640',
 'división canes',
 'forestal san miguel',
 'fountain',
 'tpt.sc.elma-2005',
 'craft beer',
 'centro medico obra social del personal de alimentos delegacion santa rosa',
 'jesús',
 'comedor santa maria de los pobres',
 'ast.ch.ae-700']

In [35]:
gdf_propiedades.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 609185 entries, 0 to 609184
Data columns (total 28 columns):
 #   Column                   Non-Null Count   Dtype   
---  ------                   --------------   -----   
 0   localidad                609185 non-null  object  
 1   id                       609185 non-null  object  
 2   ad_type                  609185 non-null  object  
 3   start_date               609185 non-null  object  
 4   end_date                 609185 non-null  object  
 5   created_on               609185 non-null  object  
 6   lat                      609185 non-null  float64 
 7   lon                      609185 non-null  float64 
 8   país                     609185 non-null  object  
 9   provincia                609185 non-null  object  
 10  rooms                    609185 non-null  float64 
 11  bedrooms                 609185 non-null  float64 
 12  bathrooms                609185 non-null  float64 
 13  surface_total            609185 non-

Se decidio suspender el enriquecimiento con los df_pois porque no encontre un metodo para realizarlo.